# Test des fonctionnalités DP_impose et K regul

In [ ]:
from trustutils import run
run.introduction("A. Gerschenfeld, Y. Gorsse")
run.description("")
run.TRUST_parameters()

In [ ]:
from trustutils import run
import os

run.reset()
dis = ["PolyMAC_P0P1NC", "PolyMAC_P0"]
pbs = ["ns", "multi"]
sources = {"dp_k_regul": "DP_Impose { dp Champ_uniforme 3 1e2 0 0 surface { surface surf2 orientation Champ_Uniforme 3 0 0 -1 } } , perte_Charge_Singuliere { dir K regul { K0 10.0 deb 5 eps 0.005*(t>0.1) } surface { Surface surf1 orientation Champ_Uniforme 3 0 0 -1 } }",
           "dp_regul": "DP_Impose { dp_regul { DP0 1e2 deb 5 eps 0.001*(t>0.1) } surface { surface surf2 orientation Champ_Uniforme 3 0 0 -1 } }"}

run.initBuildDirectory()
for nom_s, s in sources.items():
    for d in dis:
        for pb in pbs:
            run.addCaseFromTemplate(f"jdd_{pb}.data", f"{pb}/{d}/{nom_s}", {"dis" : d, "sources" : s})

run.printCases()
run.runCases()

## Description
Domaine fermé avec une frontiere interne (en gris sur la figure), et deux surfaces permettant d'imposer les termes source testés. En bleu, la régulation sur le débit, et en orange le DP imposé. Une circulation du fluide apparait, et le débit généré se régule sur la valeur cible.

![](src/domaine.png)

In [ ]:
from trustutils import plot

a = plot.Graph()
b = plot.Graph()
c = plot.Graph()
for nom_s, s in sources.items():
    for pb in pbs:
        for i, d in enumerate(dis):
            data_DP = plot.loadText(f"{pb}/{d}/{nom_s}/jdd_{pb}_pb_DP_surf2.out", skiprows=3)
            # plot K
            if nom_s == "dp_k_regul":
                data = plot.loadText(f"{pb}/{d}/{nom_s}/jdd_{pb}_pb_K_surf1.out", skiprows=3)
                a.add(data[0], data[1] ,label=f"K - {pb} - {d}")

            # plot Q
            if (i == 0 and pb == "ns" and nom_s == "dp_k_regul"): b.add(data[0], data[3] ,label="debit cible")
            if nom_s == "dp_k_regul":
                b.add(data[0],-data[2] ,label=f"{pb} - {d} - {nom_s}")
            else:
                b.add(data_DP[0],data_DP[2] ,label=f"{pb} - {d} - {nom_s}")

            # plot DP
            c.add(data_DP[0], data_DP[1], label=f"DP impose - {pb} - {d} - {nom_s}")
            # data1 = plot.loadText(f"{pb}/{d}/jdd_{pb}_P1.son", skiprows=5)
            # data2 = plot.loadText(f"{pb}/{d}/jdd_{pb}_P2.son", skiprows=5)

            # a.add(data[0], data1[1] - data2[1],label=f"DP mesure - {pb}")
a.label("Time", "Coeff de perte de charge")
b.label("Time", "Debit")
c.label("Time", "DeltaP")

# a.visu()